In [2]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

"""
Advanced 15-Minute Opening Range Breakout (ORB) Backtester
==========================================================

Strategy Overview
-----------------
Implements the **Advanced ORB strategy** (no indicators, pure price-action)
based on the first 15 minutes of trading (three 5-minute candles).

**Concept:**
1. Define the Opening Range (OR):
   - OR = first three 5-minute candles (09:15–09:30 IST)
   - ORH = highest high of these three candles
   - ORL = lowest low of these three candles

2. Identify breakout:
   - A breakout candle is one that **closes above ORH** (bullish) or **below ORL** (bearish).

3. Wait for retest:
   - After breakout, wait for a candle that **retests** the broken OR level
     (i.e., its high/low touches the ORH/ORL).

4. Confirm entry:
   - Go long if the next candle after retest **breaks the retest high**.
   - Go short if the next candle after retest **breaks the retest low**.
   - Entry executes at the **next bar open** (to simulate realistic fills).

5. Risk management:
   - Fixed absolute risk/target per trade:
     • Stop Loss (SL): ₹500 loss per trade (position-level)
     • Take Profit (TP): ₹2,000 profit per trade (position-level)
   - One trade per day per stock.
   - All open positions are forcefully **squared off** near 15:25 IST.

---

Add-Ons and Features
--------------------
**1. Intraday Leverage (Position Sizing)**
   - Supports configurable leverage (default 5×).
   - Position size = (capital_per_stock × leverage) ÷ entry_price.

**2. Direction Toggles**
   - `enable_longs`:  set to True/False to allow or block long trades.
   - `enable_shorts`: set to True/False to allow or block short trades.
   - Useful for one-sided market testing.

**3. Trailing Stop-Loss (optional)**
   - Toggle with `enable_trailing_sl`.
   - When enabled:
       • The stop follows price as it moves in your favor.
       • Distance = `trail_rupees / quantity` per share.
       • The trailing SL **tightens only** (never loosens).
       • Automatically updates each 5-minute bar:
            → For longs: new SL = max(previous_SL, highest_price - trail_distance)
            → For shorts: new SL = min(previous_SL, lowest_price + trail_distance)
       • Exit reason is labeled “SL-TRAIL” if triggered.
   - Default trail distance = ₹500 per trade (same as fixed SL).

**4. Cost Model (Groww Intraday – NSE)**
   - Includes brokerage, STT, stamp duty, exchange fees, SEBI fees, IPFT, and GST.
   - Uses:
       • Brokerage: min(₹20, 0.1% of turnover, floor ₹5 per side)
       • STT: 0.025% on sell turnover
       • Stamp Duty: 0.003% on buy turnover
       • Exchange Txn: 0.00297% each side
       • SEBI + IPFT: 0.0001% each side
       • GST: 18% on (brokerage + exchange + SEBI + IPFT)
   - Produces accurate net P&L after transaction costs.

**5. Output**
   - Saves all trades to `trades.csv` with full per-trade details:
       • entry/exit times and prices
       • direction, reason for exit, quantity, gross & net P&L
       • brokerage, turnover, leverage, and notional exposure
   - Prints a concise summary by ticker and direction, including:
       • Number of trades
       • Win rate
       • Gross, charges, and net P&L
       • Max drawdown

---

Example Configuration
---------------------
```python
CFG = Config(
    tickers=["RELIANCE.NS", "TCS.NS", "INFY.NS"],
    capital_per_stock=100_000.0,
    intraday_leverage=5.0,
    sl_rupees=500.0,
    tp_rupees=2_000.0,
    enable_longs=True,
    enable_shorts=True,
    enable_trailing_sl=True,
    trail_rupees=500.0,
)
"""

import math
import sys
from dataclasses import dataclass
from typing import List, Dict, Tuple, Optional
import pandas as pd
import numpy as np

try:
    import yfinance as yf
except Exception:
    print("Please: pip install yfinance pandas numpy")
    sys.exit(1)

# =========================
# CONFIG
# =========================
@dataclass
class Config:
    tickers: List[str]
    start_date: str = "2025-10-01"
    end_date:   str = "2025-11-01"
    interval:   str = "5m"

    capital_per_stock: float = 100_000.0   # your cash per symbol
    intraday_leverage: float = 5.0         # 5× buying power (intraday)

    sl_rupees: float = 3000.0               # absolute ₹ stop per position
    tp_rupees: float = 11000.0               # absolute ₹ target per position

    # Direction toggles
    enable_longs: bool = True
    enable_shorts: bool = False

    # Trailing SL
    enable_trailing_sl: bool = False        # <<< NEW: toggle trailing stop
    trail_rupees: float = 500.0            # <<< NEW: trailing distance per trade (₹); per-share = this/qty

    # Trading session (IST)
    or_start: str = "09:15"                # first bar inclusive
    or_end:   str = "09:30"                # exclusive -> covers 09:15, 09:20, 09:25
    entry_start: str = "09:35"             # allow entries after OR settles
    entry_end:   str = "14:45"             # last time to open new trade
    squareoff_time: str = "15:00"          # force exit if still open

    timezone: str = "Asia/Kolkata"
    out_file: str = "trades.csv"

CFG = Config(
    tickers=['ADANIENT.NS', 'ADANIPORTS.NS', 'APOLLOHOSP.NS', 'ASIANPAINT.NS', 'AXISBANK.NS', 'BAJAJ-AUTO.NS', 'BAJFINANCE.NS', 'BAJAJFINSV.NS', 'BEL.NS', 'BHARTIARTL.NS', 'CIPLA.NS', 'COALINDIA.NS', 'DRREDDY.NS', 'DUMMYTATAM.NS', 'EICHERMOT.NS', 'ETERNAL.NS', 'GRASIM.NS', 'HCLTECH.NS', 'HDFCBANK.NS', 'HDFCLIFE.NS', 'HINDALCO.NS', 'HINDUNILVR.NS', 'ICICIBANK.NS', 'ITC.NS', 'INFY.NS', 'INDIGO.NS', 'JSWSTEEL.NS', 'JIOFIN.NS', 'KOTAKBANK.NS', 'LT.NS', 'M&M.NS', 'MARUTI.NS', 'MAXHEALTH.NS', 'NTPC.NS', 'NESTLEIND.NS', 'ONGC.NS', 'POWERGRID.NS', 'RELIANCE.NS', 'SBILIFE.NS', 'SHRIRAMFIN.NS', 'SBIN.NS', 'SUNPHARMA.NS', 'TCS.NS', 'TATACONSUM.NS', 'TMPV.NS', 'TATASTEEL.NS', 'TECHM.NS', 'TITAN.NS', 'TRENT.NS', 'ULTRACEMCO.NS', 'WIPRO.NS']

)

# =========================
# GROWW CHARGES (NSE intraday)
# =========================
def groww_intraday_charges(buy_turnover: float, sell_turnover: float) -> Dict[str, float]:
    """
    Compute charges for a single round-trip intraday equity trade (NSE).
    """
    def brokerage(turnover):
        fee = min(20.0, 0.001 * turnover)
        return max(5.0, fee)  # floor ₹5

    bro_buy  = brokerage(buy_turnover)
    bro_sell = brokerage(sell_turnover)

    exch_buy  = 0.0000297 * buy_turnover
    exch_sell = 0.0000297 * sell_turnover

    sebi_buy  = 0.000001 * buy_turnover
    sebi_sell = 0.000001 * sell_turnover

    ipft_buy  = 0.000001 * buy_turnover
    ipft_sell = 0.000001 * sell_turnover

    gst_buy  = 0.18 * (bro_buy  + exch_buy  + sebi_buy  + ipft_buy)
    gst_sell = 0.18 * (bro_sell + exch_sell + sebi_sell + ipft_sell)

    stt_sell = 0.00025 * sell_turnover
    stamp_buy = 0.00003 * buy_turnover

    total = (bro_buy + bro_sell + exch_buy + exch_sell +
             sebi_buy + sebi_sell + ipft_buy + ipft_sell +
             gst_buy + gst_sell + stt_sell + stamp_buy)

    return {
        "brokerage_buy": bro_buy, "brokerage_sell": bro_sell,
        "exchange_buy": exch_buy, "exchange_sell": exch_sell,
        "sebi_buy": sebi_buy, "sebi_sell": sebi_sell,
        "ipft_buy": ipft_buy, "ipft_sell": ipft_sell,
        "gst_buy": gst_buy, "gst_sell": gst_sell,
        "stt_sell": stt_sell, "stamp_buy": stamp_buy,
        "total_charges": total
    }

# =========================
# DATA
# =========================
def fetch_5m_dataframe(ticker: str, start: str, end: str, interval: str, tz: str) -> pd.DataFrame:
    # Keep columns flat; avoid MultiIndex
    df = yf.download(
        ticker,
        start=start,
        end=end,
        interval=interval,
        auto_adjust=False,
        prepost=False,
        progress=False,
        multi_level_index=False,
        group_by="column",
    )
    if df.empty:
        return df

    # Flatten if any MultiIndex still sneaks in (future-proof)
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = [" ".join([c for c in col if c]).strip() for col in df.columns.values]

    # Ensure tz-aware in IST
    if df.index.tz is None:
        df = df.tz_localize("UTC").tz_convert(tz)
    else:
        df = df.tz_convert(tz)

    # Standardize column names
    cols_map = {c: c.title() for c in df.columns}
    df.rename(columns=cols_map, inplace=True)
    keep = [c for c in ["Open", "High", "Low", "Close", "Volume"] if c in df.columns]
    df = df[keep]
    df["Ticker"] = ticker
    return df

def session_filter(df: pd.DataFrame, date: pd.Timestamp, tz: str) -> pd.DataFrame:
    day = pd.Timestamp(date)
    day_start = pd.Timestamp(day.date().strftime("%Y-%m-%d") + " 09:15").tz_localize(tz)
    day_end   = pd.Timestamp(day.date().strftime("%Y-%m-%d") + " 15:30").tz_localize(tz)
    return df.loc[(df.index >= day_start) & (df.index <= day_end)].copy()

# =========================
# ORB LOGIC
# =========================
def _ts_on_day(day: pd.Timestamp, hhmm: str, tz: str) -> pd.Timestamp:
    return pd.Timestamp(day.date().strftime("%Y-%m-%d") + f" {hhmm}").tz_localize(tz)

def get_or_levels(day_df: pd.DataFrame, day: pd.Timestamp, or_start: str, or_end: str, tz: str) -> Tuple[float, float]:
    s = _ts_on_day(day, or_start, tz)
    e = _ts_on_day(day, or_end, tz)
    or_df = day_df.loc[(day_df.index >= s) & (day_df.index < e)]
    if len(or_df) < 3:
        return np.nan, np.nan
    return float(or_df["High"].max()), float(or_df["Low"].min())

def find_breakout(day_df: pd.DataFrame, day: pd.Timestamp, orh: float, orl: float,
                  entry_start: str, tz: str,
                  enable_longs: bool, enable_shorts: bool) -> Tuple[Optional[str], Optional[pd.Timestamp]]:
    """
    Scan after entry_start for the first allowed breakout:
    - long if Close > ORH and longs enabled
    - short if Close < ORL and shorts enabled
    Returns (direction, breakout_ts) or (None, None).
    """
    if not enable_longs and not enable_shorts:
        return None, None

    start_ts = _ts_on_day(day, entry_start, tz)
    df = day_df.loc[day_df.index >= start_ts]
    for ts, row in df.iterrows():
        if enable_longs and row["Close"] > orh:
            return "long", ts
        if enable_shorts and row["Close"] < orl:
            return "short", ts
    return None, None

def find_retest_and_trigger(day_df: pd.DataFrame, direction: str, or_level: float, breakout_ts: pd.Timestamp) -> Tuple[Optional[pd.Timestamp], float, Optional[pd.Timestamp], float]:
    """
    After breakout, wait for a retest candle that touches the broken level.
    Then place a stop-entry at the retest candle extreme.
    Execute at NEXT bar OPEN when price crosses the trigger.
    """
    post = day_df.loc[day_df.index > breakout_ts]
    for ts, row in post.iterrows():
        touched = (row["Low"] <= or_level <= row["High"])
        if not touched:
            continue
        # Retest candle found at ts
        if direction == "long":
            trigger = float(row["High"])
            nxt = day_df.loc[day_df.index > ts]
            for ts2, row2 in nxt.iterrows():
                if row2["High"] >= trigger:
                    entry_price = float(max(row2["Open"], trigger))
                    return ts, trigger, ts2, entry_price
            return ts, trigger, None, np.nan
        else:
            trigger = float(row["Low"])
            nxt = day_df.loc[day_df.index > ts]
            for ts2, row2 in nxt.iterrows():
                if row2["Low"] <= trigger:
                    entry_price = float(min(row2["Open"], trigger))
                    return ts, trigger, ts2, entry_price
            return ts, trigger, None, np.nan
    return None, np.nan, None, np.nan

# =========================
# SIMULATION (with trailing SL)
# =========================
def simulate_trade(day_df: pd.DataFrame, direction: str, entry_ts: pd.Timestamp, entry_price: float,
                   capital: float, leverage: float, sl_rupees: float, tp_rupees: float,
                   squareoff_time: str, tz: str, ticker: str,
                   enable_trailing_sl: bool, trail_rupees: float):
    if math.isnan(entry_price) or entry_ts is None:
        return None

    # Position sizing with intraday leverage
    buying_power = capital * leverage
    qty = int(buying_power // entry_price)
    if qty < 1:
        return None

    # Per-share distances
    risk_per_share   = sl_rupees / qty
    target_per_share = tp_rupees / qty
    trail_per_share  = (trail_rupees if trail_rupees is not None else sl_rupees) / qty

    # Static initial stop and target
    if direction == "long":
        static_sl = entry_price - risk_per_share
        tp = entry_price + target_per_share
        high_water = entry_price  # for trailing
        dyn_sl = static_sl
    else:
        static_sl = entry_price + risk_per_share
        tp = entry_price - target_per_share
        low_water = entry_price   # for trailing
        dyn_sl = static_sl

    exit_reason = None
    exit_ts = None
    exit_price = None

    after = day_df.loc[day_df.index >= entry_ts]
    for ts, row in after.iterrows():

        # --- Update trailing stop BEFORE checks (ratchet only) ---
        if enable_trailing_sl:
            if direction == "long":
                high_water = max(high_water, float(row["High"]))
                # Never loosen below static SL
                new_sl = max(static_sl, high_water - trail_per_share)
                dyn_sl = max(dyn_sl, new_sl)
            else:
                low_water = min(low_water, float(row["Low"]))
                # Never loosen above static SL
                new_sl = min(static_sl, low_water + trail_per_share)
                dyn_sl = min(dyn_sl, new_sl)

        # --- Exit logic (bar-based; adverse first is conservative) ---
        if direction == "long":
            # SL (static or trailed)
            if row["Low"] <= dyn_sl:
                exit_reason, exit_ts, exit_price = ("SL-TRAIL" if enable_trailing_sl else "SL",
                                                    ts, float(dyn_sl))
                break
            # TP
            if row["High"] >= tp:
                exit_reason, exit_ts, exit_price = "TP", ts, float(tp)
                break
        else:
            # SL (static or trailed)
            if row["High"] >= dyn_sl:
                exit_reason, exit_ts, exit_price = ("SL-TRAIL" if enable_trailing_sl else "SL",
                                                    ts, float(dyn_sl))
                break
            # TP
            if row["Low"] <= tp:
                exit_reason, exit_ts, exit_price = "TP", ts, float(tp)
                break

        # EOD square-off
        so = _ts_on_day(ts, squareoff_time, tz)
        if ts >= so:
            exit_reason, exit_ts, exit_price = "EOD", ts, float(row["Close"])
            break

    if exit_ts is None:
        # If still open by last bar, exit at last close
        last_ts = after.index[-1]
        exit_reason, exit_ts, exit_price = "EOD", last_ts, float(after.iloc[-1]["Close"])

    # P&L before charges + turnovers for fees
    if direction == "long":
        gross = (exit_price - entry_price) * qty
        buy_turnover  = entry_price * qty
        sell_turnover = exit_price * qty
    else:
        gross = (entry_price - exit_price) * qty
        buy_turnover  = exit_price * qty
        sell_turnover = entry_price * qty

    fees = groww_intraday_charges(buy_turnover, sell_turnover)
    net = gross - fees["total_charges"]

    return {
        "date": entry_ts.date().isoformat(),
        "ticker": ticker,
        "direction": direction,
        "entry_time": entry_ts.isoformat(),
        "entry_price": round(entry_price, 2),
        "qty": qty,
        "sl_price_initial": round(static_sl, 2),
        "tp_price": round(tp, 2),
        "sl_trailing_enabled": enable_trailing_sl,
        "trail_rupees": trail_rupees,
        "exit_time": exit_ts.isoformat(),
        "exit_price": round(exit_price, 2),
        "gross_pnl": round(gross, 2),
        "charges": round(fees["total_charges"], 2),
        "net_pnl": round(net, 2),
        "exit_reason": exit_reason,
        "buy_turnover": round(buy_turnover, 2),
        "sell_turnover": round(sell_turnover, 2),
        "cash_capital": round(capital, 2),
        "leverage": leverage,
        "notional_at_entry": round(entry_price * qty, 2),
    }

# =========================
# BACKTEST
# =========================
def run_backtest(cfg: Config) -> pd.DataFrame:
    all_trades = []

    for ticker in cfg.tickers:
        print(f"Downloading {ticker} ...")
        df = fetch_5m_dataframe(ticker, cfg.start_date, cfg.end_date, cfg.interval, cfg.timezone)
        if df.empty:
            print(f"  WARN: No data for {ticker}")
            continue

        # Unique list of session dates in this df
        dates = sorted(list({pd.Timestamp(ts).date() for ts in df.index}))

        for d in dates:
            day = pd.Timestamp(d)
            day_df = session_filter(df, day, cfg.timezone)
            if len(day_df) < 10:
                continue

            # OR levels from first three 5-min bars
            orh, orl = get_or_levels(day_df, day, cfg.or_start, cfg.or_end, cfg.timezone)
            if not np.isfinite(orh) or not np.isfinite(orl):
                continue

            # Find first allowed breakout after entry_start
            direction, bk_ts = find_breakout(
                day_df, day, orh, orl, cfg.entry_start, cfg.timezone,
                enable_longs=cfg.enable_longs,
                enable_shorts=cfg.enable_shorts
            )
            if direction is None:
                continue

            broken_level = orh if direction == "long" else orl
            rt_ts, trigger, en_ts, en_price = find_retest_and_trigger(day_df, direction, broken_level, bk_ts)
            if en_ts is None or not np.isfinite(en_price):
                continue

            # Don't enter too late
            last_entry_ts = _ts_on_day(day, cfg.entry_end, cfg.timezone)
            if en_ts > last_entry_ts:
                continue

            # Simulate one trade per day per ticker
            trade = simulate_trade(
                day_df=day_df, direction=direction,
                entry_ts=en_ts, entry_price=en_price,
                capital=cfg.capital_per_stock, leverage=cfg.intraday_leverage,
                sl_rupees=cfg.sl_rupees, tp_rupees=cfg.tp_rupees,
                squareoff_time=cfg.squareoff_time, tz=cfg.timezone,
                ticker=ticker,
                enable_trailing_sl=cfg.enable_trailing_sl,
                trail_rupees=cfg.trail_rupees
            )
            if trade:
                all_trades.append(trade)

    trades = pd.DataFrame(all_trades)
    if trades.empty:
        print("No trades generated.")
        return trades

    trades.sort_values(by=["date", "ticker", "entry_time"], inplace=True)
    trades.to_csv(cfg.out_file, index=False)
    return trades

# =========================
# METRICS
# =========================
def max_drawdown(series: pd.Series) -> float:
    cum = series.cumsum()
    peak = cum.cummax()
    dd = cum - peak
    return float(dd.min())

def summarize(trades: pd.DataFrame):
    print("\n=== OVERALL METRICS ===")
    n = len(trades)
    wins = (trades["net_pnl"] > 0).sum()
    win_rate = 100.0 * wins / n if n else 0.0
    gross = trades["gross_pnl"].sum()
    charges = trades["charges"].sum()
    net = trades["net_pnl"].sum()
    mdd = max_drawdown(trades["net_pnl"])

    print(f"Trades: {n} | Win rate: {win_rate:.1f}%")
    print(f"Gross P&L: ₹{gross:,.2f} | Charges: ₹{charges:,.2f} | Net P&L: ₹{net:,.2f}")
    print(f"Max Drawdown (net): ₹{mdd:,.2f}")

    print("\n=== BY TICKER ===")
    by_t = trades.groupby("ticker").agg(
        n=("net_pnl","count"),
        wins=("net_pnl", lambda x: (x>0).sum()),
        gross=("gross_pnl","sum"),
        charges=("charges","sum"),
        net=("net_pnl","sum"),
        win_rate=("net_pnl", lambda x: 100.0*(x>0).mean())
    ).reset_index()
    by_t["win_rate"] = by_t["win_rate"].round(1)
    print(by_t.to_string(index=False))

    print("\n=== BY DIRECTION ===")
    by_dir = trades.groupby("direction").agg(
        n=("net_pnl","count"),
        wins=("net_pnl", lambda x: (x>0).sum()),
        gross=("gross_pnl","sum"),
        charges=("charges","sum"),
        net=("net_pnl","sum"),
        win_rate=("net_pnl", lambda x: 100.0*(x>0).mean())
    ).reset_index()
    by_dir["win_rate"] = by_dir["win_rate"].round(1)
    print(by_dir.to_string(index=False))

    print("\nWrote trades to:", CFG.out_file)

# =========================
# MAIN
# =========================
if __name__ == "__main__":
    trades = run_backtest(CFG)
    if not trades.empty:
        summarize(trades)



HTTP Error 404: {"quoteSummary":{"result":null,"error":{"code":"Not Found","description":"Quote not found for symbol: DUMMYTATAM.NS"}}}

1 Failed download:
['DUMMYTATAM.NS']: YFTzMissingError('possibly delisted; no timezone found')


  WARN: No data for DUMMYTATAM.NS

=== OVERALL METRICS ===
Trades: 413 | Win rate: 37.5%
Gross P&L: ₹-58,362.83 | Charges: ₹92,558.62 | Net P&L: ₹-150,921.55
Max Drawdown (net): ₹-212,713.53

=== BY TICKER ===
       ticker  n  wins     gross  charges       net  win_rate
  ADANIENT.NS  6     1  -5200.00  1343.05  -6543.05      16.7
ADANIPORTS.NS  8     3   -994.80  1794.25  -2789.05      37.5
APOLLOHOSP.NS  6     1 -12661.50  1334.06 -13995.56      16.7
ASIANPAINT.NS  7     2  -2693.40  1568.98  -4262.37      28.6
  AXISBANK.NS  9     2  -7133.10  2017.31  -9150.41      22.2
BAJAJ-AUTO.NS 10     6   2359.00  2229.82    129.18      60.0
BAJAJFINSV.NS  6     2  -4429.16  1345.06  -5774.20      33.3
BAJFINANCE.NS  8     3   4239.30  1796.74   2442.55      37.5
       BEL.NS  6     4   1917.06  1347.81    569.25      66.7
BHARTIARTL.NS 11     6  11484.75  2470.15   9014.60      54.5
     CIPLA.NS  7     1  -8322.90  1567.97  -9890.87      14.3
 COALINDIA.NS  9     4  -3334.22  2019.98  -53